# 5. 실시간 데이터 기반 경고 메시지 알림 구현

* SparkSession을 생성하고 소켓으로 데이터를 수신하도록 스트리밍 설정을 하고 원하는 형태의 데이터만 다룰 수 있도록 합니다.
* 해당 데이터를 window와 watermarking을 다루는 방법을 익힙니다.
* 특정 경고 단어 **error, fail, warning** 가 발생할 때 알림을 출력하는 시스템

- **1분 윈도우 내 특정 단어 발생 횟수**: 각 `1분` 간격의 고정 윈도우에서 특정 단어 (`error`, `fail`, `warning`)가 발생하는 횟수를 집계합니다.
  
- **발생 빈도 기준**: 단어가 `1분 윈도우` 내에서 **3회 이상** 발생한 경우에만 알림을 출력합니다.





### 입력 예시
- fail fail warning
- error warning fail
- fail error warning
- 1분 후
- warning fail fail
- error fail warning


###  출력 예시 

**ALERT**: `'fail'` occurred **4 times** in the window `2024-11-09 12:02:00` to `2024-11-09 12:03:00`  
**ALERT**: `'warning'` occurred **3 times** in the window `2024-11-09 12:02:00` to `2024-11-09 12:03:00`

**Current Batch Data:**

| window                                   | word    | count |
|------------------------------------------|---------|-------|
| 2024-11-09 12:02:00 - 2024-11-09 12:03:00 | fail    | 4     |
| 2024-11-09 12:02:00 - 2024-11-09 12:03:00 | warning | 3     |
| 2024-11-09 12:02:00 - 2024-11-09 12:03:00 | error   | 2     |

---

**ALERT**: `'fail'` occurred **3 times** in the window `2024-11-09 12:03:00` to `2024-11-09 12:04:00`

**Current Batch Data:**

| window                                   | word    | count |
|------------------------------------------|---------|-------|
| 2024-11-09 12:03:00 - 2024-11-09 12:04:00 | fail    | 3     |
| 2024-11-09 12:03:00 - 2024-11-09 12:04:00 | error   | 1     |
| 2024-11-09 12:03:00 - 2024-11-09 12:04:00 | warning | 2     |



### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, current_timestamp, window, expr

# SparkSession 생성
# 기존에 열려 있는 Spark 세션이 있다면 중지하고, 새로운 세션을 생성합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = SparkSession.builder \
    .appName("StructuredStreamingAlertSystem") \
    .getOrCreate()

In [ ]:
# 소켓 스트림을 통한 데이터 수신 설정
# 'localhost'의 '9999' 포트에서 소켓 데이터를 실시간으로 수신하여 Spark DataFrame에 로드합니다.
# 'nc -lk 9999' 명령어로 소켓 서버를 시작한 후, 터미널에서 텍스트 입력이 가능합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
lines = spark._____________ \
    ._____________("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()


In [ ]:
# 타임스탬프 열 추가 및 단어 분리
# 수신한 각 라인에 현재 시간 타임스탬프를 추가하여 이벤트 시간 기반 처리가 가능하도록 합니다.
# explode와 split 함수를 사용해 각 줄을 공백(" ")으로 나누고, 개별 단어를 행으로 분리합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.current_timestamp.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html
lines_with_timestamp = lines._____________("timestamp", _____________())
words = lines_with_timestamp.select(_____________(split(lines.value, " ")).alias("word"), "timestamp")

In [ ]:
# 경고 조건 단어 설정
# 특정 단어 리스트 ('error', 'fail', 'warning')를 설정하고, 해당 단어가 포함된 경우만 필터링합니다.
# expr을 통해 필터링을 수행하여, 경고 메시지를 발생시키고자 하는 특정 단어만 집계에 포함합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html
alert_words = ["error", "fail", "warning"]
alerts = words.filter(_____________("word IN ('error', 'fail', 'warning')"))


In [ ]:
# 워터마크 및 고정 윈도우 설정
# 워터마크 설정으로 2분이 지난 오래된 데이터는 집계에서 제외합니다.
# groupBy를 통해 1분 고정 윈도우와 단어별로 그룹핑하여 발생 횟수를 집계합니다.
# 윈도우를 사용하여 1분 동안 발생한 단어 빈도를 집계하고, 이를 10초 간격으로 확인합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withWatermark.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html
alertCounts = alerts \
    ._____________("timestamp", "2 minutes") \
    .groupBy(_____________("timestamp", "1 minute"), "word") \
    .count()

In [ ]:
# 경고 메시지 출력 함수
# foreachBatch에서 각 배치에 대해 이 함수가 호출됩니다.
def alert_func(df, epoch_id):
    # 필터링 조건: 'count' 값이 3 이상인 단어만 선택
    # 1분 윈도우 내에서 특정 단어가 3회 이상 발생한 경우에만 경고 메시지를 출력합니다.
    df_filtered = df.filter("count >= 3")
    
    # 경고 조건을 만족하는 단어가 있을 경우 콘솔에 경고 메시지 출력
    for row in df_filtered.collect():
        print(f"ALERT: '{row['word']}' occurred {row['count']} times in the window {row['window']}")
    
    # 현재 배치의 전체 집계 데이터를 출력하여 경고 조건을 만족하지 않는 데이터도 확인 가능
    print("Current Batch Data:")
    df.show(truncate=False)

In [ ]:
# foreachBatch를 사용하여 각 배치에서 alert_func 호출
# foreachBatch(alert_func)로 각 배치별로 alert_func을 호출하여, 집계된 결과를 기반으로 경고 조건을 검사합니다.
# outputMode("update")는 변경된 데이터만 업데이트하여 출력합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.DataStreamWriter.foreachBatch.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.trigger.html
alert_query = alertCounts._____________ \
    ._____________(alert_func) \
    ._____________("update") \
    ._____________(processingTime="10 seconds") \
    .start()

# 스트리밍 쿼리를 유지하여 계속 실행되도록 합니다.
# query.awaitTermination()을 통해 쿼리가 종료될 때까지 프로그램이 유지됩니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.StreamingQuery.awaitTermination.html
alert_query.awaitTermination()